In [1]:
from bertopic import BERTopic

/home/ricorenzo/.pyenv/versions/3.10.6/envs/speeches-UN/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ricorenzo/.pyenv/versions/3.10.6/envs/speeches-UN/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ricorenzo/.pyenv/versi

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
df = pd.read_csv('../raw_data/all_speeches_preprocessed.csv')
df = df.drop(['Unnamed: 0.1','Unnamed: 0','Post', 'Unnamed: 6'], axis=1)
df = df.dropna()

In [59]:
df.head(5)

,iso,year,session,year_iso,speeches,index,Year,Session,ISO Code,Country,Name of Person Speaking,order,cleaned_speeches,preprocessed_speech,preprocessed_speeches_joined
0,BEL,1950,77,1950_BEL,Never before have men throughout the world mor...,10375.0,1950.0,5.0,BEL,Belgium,Mr. VAN ZEELAND,10375.0,never before have men throughout the world mor...,"['never', 'man', 'throughout', 'world', 'ferve...",never man throughout world fervently unanimous...
1,BLR,1950,77,1950_BLR,It is now five years since the United Nations ...,10397.0,1950.0,5.0,BLR,Byelorussian Soviet Socialist Republic,Mr. KISELEV,10397.0,it is now five years since the united nations ...,"['five', 'year', 'since', 'united', 'nations',...",five year since united nations charter lay fun...
2,FRA,1950,77,1950_FRA,"At the conclusion of this general discussion, ...",10391.0,1950.0,5.0,FRA,France,Mr. R. SCHUMAN,10391.0,at the conclusion of this general discussion i...,"['conclusion', 'general', 'discussion', 'much'...",conclusion general discussion much eloquence e...
3,PAK,1950,77,1950_PAK,May I be permitted to congratulate the General...,10386.0,1950.0,5.0,PAK,Pakistan,Sir Mohammad ZAFRULLA KHAN,10386.0,may i be permitted to congratulate the general...,"['may', 'permit', 'congratulate', 'general', '...",may permit congratulate general assembly choic...
4,TUR,1950,77,1950_TUR,The era inaugurated by the creation of the Uni...,10382.0,1950.0,5.0,TUR,Turkey,Mr. SARPER,10382.0,the era inaugurated by the creation of the uni...,"['era', 'inaugurate', 'creation', 'united', 'n...",era inaugurate creation united nations one cri...


In [60]:
def bertopic(texts):
    topic_model = BERTopic()
    topic_model.fit(texts)
    #topics, probs = topic_model.fit_transform(texts)

    return topic_model

topic_model = BERTopic(top_n_words = 10,
        n_gram_range = (1, 3),
        # embedding_model = None,
        # vectorizer_model = None,
        min_topic_size = 5,
        nr_topics = 'HDBSCAN'
        )


topics, probs = topic_model.fit_transform(df.preprocessed_speeches_joined)
df['topic_label'] = topic_model.topics_

In [67]:
df['topic_label'].nunique()

91

In [5]:
texts = list(df['preprocessed_speeches_joined'].dropna())

In [8]:
speechs_bertopic = topic_model.get_document_info(texts)
speechs_bertopic.head(3)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,never man throughout world fervently unanimous...,-1,-1_united_international_nations_united nations,"[united, international, nations, united nation...",[like first express sir personal satisfaction ...,united - international - nations - united nati...,0.00000,False
1,five year since united nations charter lay fun...,0,0_united_international_nations_united nations,"[united, international, nations, united nation...",[attention world rivet session united nations ...,united - international - nations - united nati...,0.94701,False
2,conclusion general discussion much eloquence e...,0,0_united_international_nations_united nations,"[united, international, nations, united nation...",[attention world rivet session united nations ...,united - international - nations - united nati...,0.81528,False


In [10]:
def dataframe_to_dictionary(df):
    topics_dictionary = {speechs_bertopic['Topic']: speechs_bertopic['Name'] for _, speechs_bertopic in df.iterrows()}
    return topics_dictionary

In [11]:
topics_dict = dataframe_to_dictionary(speechs_bertopic)
topics=list(topics_dict.values())
topics_df = pd.DataFrame([each.split('_') for each in topics])

topics_df_no_index = topics_df.iloc[:,1:]
topics_df_no_index['sentence'] = topics_df_no_index[1] + ' ' + topics_df_no_index[2] + ' ' + topics_df_no_index[3] + ' ' + topics_df_no_index[4]

topics_df_no_index

from sentence_transformers import SentenceTransformer
# Calculate similarities between sentences
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-mpnet-base-v2')


In [12]:
# Split text into sentences
# Embed sentences
embeddings = model.encode(topics_df_no_index['sentence'])
embeddings

array([[ 0.01960675,  0.03746051,  0.02125376, ..., -0.02053328,
         0.02722503,  0.02291734],
       [ 0.01960675,  0.03746051,  0.02125376, ..., -0.02053328,
         0.02722503,  0.02291734],
       [ 0.0486006 , -0.02274154, -0.01944033, ..., -0.03652029,
        -0.00833334,  0.00490806],
       ...,
       [-0.02010631, -0.0413814 , -0.03719605, ..., -0.03366031,
        -0.03135264, -0.00287349],
       [ 0.0187493 ,  0.04542708,  0.01066927, ..., -0.03489419,
         0.03128538,  0.00873028],
       [ 0.0115528 ,  0.07923269,  0.004413  , ...,  0.02965353,
        -0.0131054 , -0.02754805]], dtype=float32)

In [13]:
from sklearn.decomposition import PCA

In [14]:
embedding_df = pd.DataFrame(embeddings)
embedding_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.019607,0.037461,0.021254,0.005744,-0.000277,0.001174,0.128796,-0.019445,0.051629,0.018941,...,-0.057745,0.041775,-0.016132,0.012740,-0.026379,-0.110636,-0.001688,-0.020533,0.027225,0.022917
1,0.019607,0.037461,0.021254,0.005744,-0.000277,0.001174,0.128796,-0.019445,0.051629,0.018941,...,-0.057745,0.041775,-0.016132,0.012740,-0.026379,-0.110636,-0.001688,-0.020533,0.027225,0.022917
2,0.048601,-0.022742,-0.019440,-0.057427,0.024745,-0.011132,-0.025792,-0.023738,0.058789,0.039799,...,-0.024282,-0.061210,0.006504,0.025778,-0.027838,-0.027053,0.008918,-0.036520,-0.008333,0.004908
3,0.009913,-0.019235,0.020752,-0.060168,0.002029,-0.002851,0.066649,-0.035710,0.091034,0.023104,...,-0.062275,-0.021151,0.001058,0.035059,-0.002431,-0.055324,-0.018428,-0.024289,0.018927,-0.005421
4,0.016861,0.112931,0.021521,0.012802,-0.036480,-0.039181,0.003147,0.018685,0.025166,0.066716,...,-0.066315,0.016531,0.015001,0.024102,0.027300,-0.034945,0.020326,0.028878,0.033900,0.037002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0.043173,-0.041430,0.012603,-0.029735,-0.026030,-0.018419,0.011341,-0.001640,-0.020232,0.036588,...,-0.057252,-0.009032,0.033262,0.039749,0.012935,-0.056057,-0.001603,0.028108,-0.041714,0.008122
136,0.015668,0.045421,0.019995,0.025656,0.016795,0.023699,0.103695,-0.030392,0.064917,0.001700,...,-0.055418,0.026144,0.008625,-0.003141,-0.036164,-0.109305,0.003274,-0.026898,0.024354,0.009509
137,-0.020106,-0.041381,-0.037196,0.001809,0.014542,0.024411,0.071282,-0.001044,-0.047188,-0.017051,...,-0.045046,-0.004876,0.097277,-0.003466,0.018894,0.006934,-0.022922,-0.033660,-0.031353,-0.002873
138,0.018749,0.045427,0.010669,-0.036944,0.012186,-0.002741,0.024760,0.000134,0.040910,-0.013285,...,-0.039460,0.029918,0.029626,0.032862,0.015115,-0.053432,-0.032109,-0.034894,0.031285,0.008730


In [15]:
pca = PCA(n_components=3)
pca_df = pca.fit_transform(embedding_df)

In [16]:
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=24)
kmean.fit(pca_df)

KMeans(n_clusters=24)

In [17]:
embedding_df['label'] = kmean.labels_
topics_df['label'] = kmean.labels_
topics_df.to_csv('~/code/renzorico/speeches-at-UN/raw_data/topics')


In [32]:
topics_df

,0,1,2,3,4,label
0,-1,united,international,nations,united nations,16
1,0,united,international,nations,united nations,16
2,11,korea,republic korea,korean,democratic people republic,22
3,6,soviet,soviet union,state,union,22
4,134,argentine,international,peace,principle,1
...,...,...,...,...,...,...
135,98,disarmament,malaya,one,world,11
136,85,united,assembly,nations,united nations,16
137,138,marshall,island,marshall islands,islands,18
138,24,united,state,world,country,20


In [ ]:
# inertias = []
# ks = range(1,100)

# for k in ks:
#     km_test = KMeans(n_clusters=k).fit(pca_df)
#     inertias.append(km_test.inertia_)

# plt.plot(ks, inertias)
# plt.xlabel('k cluster number')

In [ ]:
topic_model.get_document_info(texts).to_csv('../raw_data/bertopics.csv')

In [18]:
speechs_bertopic['Topic']

0        -1
1         0
2         0
3        -1
4         0
         ..
10393     0
10394     0
10395    89
10396    -1
10397     0
Name: Topic, Length: 10398, dtype: int64

In [19]:
topics_df.loc[topics_df.label == 1]

,0,1,2,3,4,label
4,134,argentine,international,peace,principle,1
44,71,colombian,drug,peace,world,1
54,72,develop,development,international,develop country,1
63,51,development,country,international,world,1
86,76,gabonese,development,international,africa,1
88,62,philippine,development,united,nations,1
89,49,world,economic,country,great,1
103,120,togolese,peace,development,country,1
127,1,pandemic,covid,un,global,1
